In [73]:
import torch
import torch.nn.functional as Fun
import spacy
import en_core_web_sm

In [3]:
x = torch.arange(10,20)
x

tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

#### One Hot Encoding

In [21]:
# One hot encoding using pytorchs functional class
y = Fun.one_hot(x,num_classes=20) # doesnt implement in similar way like sklearns ohe does
y

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
ohe = OneHotEncoder()
y2 = ohe.fit_transform(x.reshape(-1,1))
y2.toarray() # since the return value is a sparse array we need to convert it

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [24]:
import pandas as pd

In [56]:
df = pd.DataFrame({"text":["people watch campusx","campusx watch campusx","people write comment","campusx write comment"],"output":[1,1,0,0]})

In [57]:
df.head()

,text,output
0,people watch campusx,1
1,campusx watch campusx,1
2,people write comment,0
3,campusx write comment,0


In [84]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Rejoin tokens
    return ' '.join(tokens)


[nltk_data] Downloading package punkt to /home/satya-
[nltk_data]     tt0492/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/satya-
[nltk_data]     tt0492/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/satya-
[nltk_data]     tt0492/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [86]:
df["text"] = df["text"].apply(preprocess_text)
df.head()

,text,output
0,people watch campusx,1
1,campusx watch campusx,1
2,people write comment,0
3,campusx write comment,0


In [107]:
# One Hot Encoding (individual tokens)

tokens = df["text"].apply(lambda x : x.split())
tokens

0     [people, watch, campusx]
1    [campusx, watch, campusx]
2     [people, write, comment]
3    [campusx, write, comment]
Name: text, dtype: object

In [96]:
unique_tokens = list(set([j for i in tokens for j in i]))
unique_tokens

['people', 'watch', 'campusx', 'write', 'comment']

In [101]:
ohe = OneHotEncoder(sparse_output=False)
ohe_enc = ohe.fit_transform([[token] for token in unique_tokens])
ohe_enc

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]])

In [111]:
# now map to each unique token
mapper = dict(zip(unique_tokens,ohe_enc.tolist()))
mapper

{'people': [0.0, 0.0, 1.0, 0.0, 0.0],
 'watch': [0.0, 0.0, 0.0, 1.0, 0.0],
 'campusx': [1.0, 0.0, 0.0, 0.0, 0.0],
 'write': [0.0, 0.0, 0.0, 0.0, 1.0],
 'comment': [0.0, 1.0, 0.0, 0.0, 0.0]}

In [112]:
tokens

0     [people, watch, campusx]
1    [campusx, watch, campusx]
2     [people, write, comment]
3    [campusx, write, comment]
Name: text, dtype: object

In [113]:
encoded_tokens = [[mapper[token] for token in sent] for sent in tokens]
encoded_tokens

[[[0.0, 0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 1.0, 0.0],
  [1.0, 0.0, 0.0, 0.0, 0.0]],
 [[1.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 1.0, 0.0],
  [1.0, 0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0],
  [0.0, 1.0, 0.0, 0.0, 0.0]],
 [[1.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0],
  [0.0, 1.0, 0.0, 0.0, 0.0]]]

In [115]:
data_to_ohe_map = list(zip(tokens,encoded_tokens))
data_to_ohe_map

[(['people', 'watch', 'campusx'],
  [[0.0, 0.0, 1.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 1.0, 0.0],
   [1.0, 0.0, 0.0, 0.0, 0.0]]),
 (['campusx', 'watch', 'campusx'],
  [[1.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 1.0, 0.0],
   [1.0, 0.0, 0.0, 0.0, 0.0]]),
 (['people', 'write', 'comment'],
  [[0.0, 0.0, 1.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 1.0],
   [0.0, 1.0, 0.0, 0.0, 0.0]]),
 (['campusx', 'write', 'comment'],
  [[1.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 1.0],
   [0.0, 1.0, 0.0, 0.0, 0.0]])]

### Bag of words

In [116]:
from sklearn.feature_extraction.text import CountVectorizer

In [137]:
cv = CountVectorizer() # try max_features hyperparameter which can be used to trim of rare features
bow = cv.fit_transform(df["text"])

In [138]:
df["text"]

0     people watch campusx
1    campusx watch campusx
2     people write comment
3    campusx write comment
Name: text, dtype: object

In [139]:
cv.vocabulary_ #indicates index values

{'people': 2, 'watch': 3, 'campusx': 0, 'write': 4, 'comment': 1}

In [140]:
cv.get_feature_names_out()

array(['campusx', 'comment', 'people', 'watch', 'write'], dtype=object)

In [141]:
bow.toarray()

array([[1, 0, 1, 1, 0],
       [2, 0, 0, 1, 0],
       [0, 1, 1, 0, 1],
       [1, 1, 0, 0, 1]])